In [ ]:
import pandas as pd
predictions = pd.read_csv('../data/processed/predictions.csv')
optimal_prices = pd.read_csv('../data/processed/optimal_prices.csv')

In [ ]:
import matplotlib.pyplot as plt
# Plot the demand curve for this SKU
sku_results = predictions[predictions['SKU'] == sku]
plt.plot(sku_results['price'], sku_results['predicted_sales_xgboost'], label=sku)

    # Add labels and legend to the plot
plt.xlabel('Price')
plt.ylabel('Predicted Sales')
plt.legend()

    # Show the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt

def perform_inference(train_data, test_data):
    # Fit the XGBoost model
    xgboost_model = XGBRegressor()
    xgboost_model.fit(train_data.drop(columns=['SKU', 'sales']), train_data['sales'])

    # Group test data by SKU
    grouped = test_data.groupby('SKU')

    # Create an empty dataframe to store the results
    results = pd.DataFrame(columns=['SKU', 'price', 'predicted_sales_xgboost'])

    # Perform inference for each SKU in test data
    for sku, group_data in grouped:
        print(f"Performing inference for SKU: {sku}")

        # Select one row from group data (one set of feature values for this SKU)
        feature_values = group_data.iloc[0]

        # Set the base price from data
        base_price = feature_values['price']

        # Define the price points
        price_points = [base_price, 
                        base_price * 0.9,   # base_price - 10%
                        base_price * 0.85,  # base_price - 15%
                        base_price * 1.1,   # base_price + 10%
                        base_price * 1.15]  # base_price + 15%

        # For each price point, use the trained model to predict sales and print the result
        for price in price_points:
            # Change the price while keeping other features constant
            test_features = feature_values.copy()
            test_features['price'] = price

            # Remove the SKU field to match the input shape expected by the model
            test_features = test_features.drop(['SKU', 'sales'])
 
            # Predict sales using the XGBoost model and append the results to the dataframe
            predicted_sales_xgboost = xgboost_model.predict(test_features.values.reshape(1, -1))[0]
            results = results.append({'SKU': sku, 'price': price, 'predicted_sales_xgboost': predicted_sales_xgboost}, ignore_index=True)

        # Plot the demand curve for this SKU
        sku_results = results[results['SKU'] == sku]
        plt.plot(sku_results['price'], sku_results['predicted_sales_xgboost'], label=sku)

    # Add labels and legend to the plot
    plt.xlabel('Price')
    plt.ylabel('Predicted Sales')
    plt.legend()

    # Show the plot
    plt.show()

    return results

In [ ]:
import matplotlib.pyplot as plt

def plot_demand_curve(predictions, sku):
    # Select the predictions for the given SKU
    sku_predictions = predictions[predictions['SKU'] == sku]

    # Plot the predicted sales against the price points
    plt.plot(sku_predictions['price'], sku_predictions['predicted_sales_xgboost'], label=sku)

    # Add labels and legend to the plot
    plt.xlabel('Price')
    plt.ylabel('Predicted Sales')
    plt.legend()

    # Show the plot
    plt.show()

In [8]:
# load datasets
import pandas as pd
predictions = pd.read_csv("../data/processed/predictions.csv")
optimal_prices = pd.read_csv("../data/processed/optimal_prices.csv")
amazon = pd.read_csv("../data/raw/Amazon_Sale_Report.csv")
pl = pd.read_csv("../data/raw/PLMarch2021.csv")

predictions = predictions.merge(amazon[['SKU', 'Category']], on='SKU', how='left')

/var/folders/qd/2qwtnqg57r915dp6p3dq762c0000gn/T/ipykernel_11042/2272987174.py:5: DtypeWarning:

Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.



In [9]:
# rename sku column to match optimal prices
pl.rename(columns={'Sku': 'SKU'}, inplace=True)

In [10]:
# write and an underscore on whitespace for every column in pl dataframe
pl.columns = pl.columns.str.replace(' ', '_')

In [11]:

pl.Category.rename({'Kurta': 'kurta', 'Kurta Set':'kurta', 'Gown':'Ethnic Dress', 'Tops':'Set','Nill':'Top' }, inplace=True)




In [12]:
#print(pl.head().to_string(index=False))
competitor_columns = ['Amazon_MRP', 'Flipkart_MRP', 'Ajio_MRP', 'Limeroad_MRP', 'Myntra_MRP', 'Paytm_MRP', 'Snapdeal_MRP']
from src.data.utils import Utils
# convert to int the competitor_columns in the pl dataframe
for col in competitor_columns:
    pl[col] = pl[col].apply(Utils.convert_to_int)
    

#avg_prices = [pl[col].mean() for col in competitor_columns]

In [13]:
avg_prices = [pl[col].mean() for col in competitor_columns]

In [14]:

import gradio as gr

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import plotly.graph_objs as go



    
unique_skus = sorted(predictions['SKU'].unique())
unique_category = sorted(pl['Category'].unique())

def plot_demand_curve_gradio(sku):
    # Load the predictions dataframe
    #predictions = pd.read_csv('../data/processed/predictions.csv')

    # Select the predictions for the given SKU
    sku_predictions = predictions[predictions['SKU'] == sku]

    # Create a scatter plot of the predicted sales against the price points
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=sku_predictions['price'], y=sku_predictions['predicted_sales_xgboost'], mode='lines+markers'))


    # Set the plot title and axis labels
    fig.update_layout(title=f'Demand Curve for SKU {sku}', xaxis_title='Price', yaxis_title='Predicted Sales')
    fig.update_xaxes(title_font=dict(size=18), tickfont=dict(size=14))
    fig.update_yaxes(title_font=dict(size=18), tickfont=dict(size=14))

    # Set the legend font size and position
    fig.update_layout(legend=dict(font=dict(size=14), yanchor='top', y=0.99, xanchor='left', x=0.01), legend_title_text='SKU')

    # Set the plot background color and grid lines
    fig.update_layout(plot_bgcolor='white', xaxis=dict(showgrid=True, gridcolor='lightgray'), yaxis=dict(showgrid=True, gridcolor='lightgray'))


    # Return the plot as an HTML div
    return gr.update(value=fig, visible=True)

def plot_optimal_prices_gradio(sku):
    # Load the optimal prices dataframe
    #optimal_prices = pd.read_csv('../data/processed/optimal_prices.csv')

    # Select the optimal prices for the given SKU
    sku_optimal_prices = optimal_prices[optimal_prices['SKU'] == sku]

    # Create a bar chart of the optimal prices at different price points
    fig = go.Figure()
    fig.add_trace(go.Bar(x=['Base Price', '-10%', '-15%', '+10%', '+15%'], y=sku_optimal_prices['optimal_prices'], name=sku))

    # Set the plot title and axis labels
    fig.update_layout(title=f'Optimal Prices for SKU {sku}', xaxis_title='Price Point', yaxis_title='Optimal Price', font=dict(size=14))
    fig.update_xaxes(title_font=dict(size=18), tickfont=dict(size=14))
    fig.update_yaxes(title_font=dict(size=18), tickfont=dict(size=14))

    # Set the plot background color and grid lines
    fig.update_layout(plot_bgcolor='white', xaxis=dict(showgrid=True, gridcolor='lightgray'), yaxis=dict(showgrid=True, gridcolor='lightgray'))

    # Return the plot as an HTML div
    return gr.update(value=fig, visible=True)

def plot_competitors_prices_gradio(category):

    sku_competitors_results = pl[pl['Category'] == category]


    # Calculate average prices for each competitor within the selected category
    competitor_columns = ['Amazon_MRP', 'Flipkart_MRP', 'Ajio_MRP', 'Limeroad_MRP', 'Myntra_MRP', 'Paytm_MRP', 'Snapdeal_MRP']
    avg_prices = [sku_competitors_results[col].median() for col in competitor_columns]

    # Create a bar chart of the average competitors prices
    fig = go.Figure()
    fig.add_trace(go.Bar(x=competitor_columns, y=avg_prices, name=category))

    # set the plot title and axis labels
    fig.update_layout(title=f'Competitors Prices for Category {category}', xaxis_title='Competitor', yaxis_title='Median Price', font=dict(size=14))
    fig.update_xaxes(title_font=dict(size=18), tickfont=dict(size=14))
    fig.update_yaxes(title_font=dict(size=18), tickfont=dict(size=14))

    # Set the plot background color and grid lines
    fig.update_layout(plot_bgcolor='white', xaxis=dict(showgrid=True, gridcolor='lightgray'), yaxis=dict(showgrid=True, gridcolor='lightgray'))

    # return the plot as an HTML div
    return gr.update(value=fig, visible=True)




with gr.Blocks() as demo:
    gr.Markdown("""
    # Relu Demand Curve & Optimal Pricing Dashboard 💰 #
    
   Version: 1.0.0
    
    Enter Values below then click Predict and Pricing Buttons to see results.  
    
    """)
    with gr.Row(equal_height=True):
      
        sku = gr.Dropdown(
            label="Select SKU",
            choices=unique_skus,
            value=lambda: random.choice(unique_skus),
        )

    with gr.Row(equal_height=True):

        with gr.Column():
            plot = gr.Plot()
            with gr.Row():
                predict_btn = gr.Button(value="Predict")
            predict_btn.click(
                plot_demand_curve_gradio,
                inputs=[sku],

                outputs= plot

            )
        with gr.Column():

            plot = gr.Plot()
            with gr.Row():

                optimal_price_btn = gr.Button(value="Optimal Price")
                optimal_price_btn.click(
                    plot_optimal_prices_gradio,
                    inputs=[sku],
                    outputs=plot

                )
        # return plot_comeptitors_prices_gradio(sku) as a plot for every sku
    with gr.Row(equal_height=True):
            category = gr.Dropdown(
            label="Select Category",
            choices=unique_category,
            value=lambda: random.choice(unique_category))


            with gr.Column():
                plot = gr.Plot()
                with gr.Row():
                    competitor_btn = gr.Button(value="Competitors Prices")
                    competitor_btn.click(
                        plot_competitors_prices_gradio,
                        inputs=[category],
                        outputs=plot)



demo.launch(share=True)



Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://a390ec7545a0d7bbc0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
